# Entregable 2 modelo de clasificación usando framework

## Dataset Utilizado

**Nombre del Dataset:** SMS Spam Collection Dataset

**Enlace al Dataset:** [SMS Spam Collection Dataset](https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset)

**Descripción de los Datos:**

- Cantidad de Registros/Muestras: El dataset contiene un total de 5,572 mensajes de texto etiquetados como spam o no spam ("ham").

- Número de Características: Las características se derivan de la representación de texto en forma de vectores de términos (TF-IDF), por lo que el número de características depende de la dimensionalidad del espacio de términos. En este caso, se han utilizado un máximo de 10,000 términos, lo que significa que cada mensaje se representa mediante un vector de características de longitud 10,000.

- Número de Clases de Salida: El problema es de clasificación binaria, por lo que hay dos clases de salida: "spam" y "no spam" (o "ham").

## Tipo de Problema

Este proyecto aborda un problema de **clasificación**. El objetivo es predecir si un mensaje de texto es spam ("spam") o no spam ("ham") en función de su contenido y características.

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('wordnet')
from sklearn.datasets import load_files
nltk.download('stopwords')
nltk.download('omw-1.4')
import pickle
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from sklearn import metrics
from sklearn.naive_bayes import GaussianNB

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\elian\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\elian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\elian\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Preprocesamiento de datos


En esta sección, se realiza el preprocesamiento de los mensajes de texto. Cada mensaje se somete a una serie de transformaciones, que incluyen la eliminación de caracteres especiales, eliminación de caracteres individuales, conversión a minúsculas y lematización (reducción de palabras a su forma base). Los mensajes preprocesados se almacenan en la lista documents.

In [2]:
mssgdata = pd.read_csv('spam.csv', encoding='latin-1')
X, y = mssgdata.v2, mssgdata.v1

documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # quita todos los caracteres especiales
    document = re.sub(r'\W', ' ', str(X[sen]))

    # quita todos los caracteres solos
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Quita los caracteres solos del inicio
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Sustituye espacios multiples por unicos
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Quita prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # minusculas
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

Después se utiliza la técnica de vectorización de texto para convertir los mensajes de texto preprocesados en una representación numérica. El método CountVectorizer se utiliza para crear vectores de características a partir de palabras en los mensajes. Se especifican varios parámetros, como el número máximo de características (términos), el rango de n-gramas y las palabras stopwords (palabras comunes que se eliminan).

In [3]:
vectorizer = CountVectorizer(max_features=10000, min_df=1, max_df=0.6, ngram_range=(1,2), stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

Después de la vectorización, se aplica la transformación TF-IDF (Term Frequency-Inverse Document Frequency) a los datos. Esta transformación pondera la importancia de las palabras en los mensajes. El resultado se almacena nuevamente en la matriz X.
Por último, se dividen los datos en conjuntos de entrenamiento y prueba. El 80% de los datos se utiliza para entrenar los modelos de clasificación, y el 20% se reserva para evaluar el rendimiento de los modelos.

In [4]:
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Modelo 1: Naive Bayes

In [5]:
model = GaussianNB()

model.fit(X_train, y_train)
print(model)

expected = y_test
predicted = model.predict(X_test)

print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))
print(accuracy_score(expected,predicted))

GaussianNB()
              precision    recall  f1-score   support

         ham       0.98      0.94      0.96       949
        spam       0.71      0.89      0.79       166

    accuracy                           0.93      1115
   macro avg       0.84      0.91      0.87      1115
weighted avg       0.94      0.93      0.93      1115

[[889  60]
 [ 19 147]]
0.9291479820627803


**Métricas de rendimiento del modelo Naive Bayes GaussianNB en el conjunto de prueba**

 Precisión se refiere a la proporción de predicciones positivas que fueron correctas, recall representa la proporción de ejemplos positivos que fueron correctamente identificados, y la puntuación F1 es una medida que combina precisión y recall. 
 
 El modelo tiene una alta precisión para la clase "ham" (no spam) del 98%, pero una precisión más baja del 71% para la clase "spam". El recall para "ham" es del 94% y para "spam" es del 89%, lo que indica que el modelo tiende a identificar mejor los mensajes "ham". El accuracy del modelo es del 92.91%, lo que significa que clasifica correctamente el 92.91% de los mensajes en el conjunto de prueba. 
 
 La matriz de confusión muestra los detalles de las predicciones, donde 889 mensajes "ham" se clasificaron correctamente, 60 se clasificaron erróneamente como "spam", 19 mensajes "spam" se clasificaron erróneamente como "ham" y 147 mensajes "spam" se clasificaron correctamente. En general, el modelo tiene un buen rendimiento en la identificación de mensajes "ham" pero puede mejorar en la identificación de mensajes "spam".

### Modelo 2: Random Forest

Se crea un modelo de clasificación utilizando el algoritmo Random Forest Classifier. Se configura con 1,000 árboles (estimadores) en el bosque (n_estimators=1000) y se establece una semilla aleatoria (random_state=0) para garantizar que los resultados sean reproducibles. Luego, se entrena el modelo utilizando los datos de entrenamiento (X_train y y_train), donde X_train contiene los vectores numéricos de características de los mensajes de texto y y_train contiene las etiquetas de clasificación (spam o no spam). A continuación, se realiza una predicción en el conjunto de prueba (X_test).

In [6]:
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[949   0]
 [ 26 140]]
              precision    recall  f1-score   support

         ham       0.97      1.00      0.99       949
        spam       1.00      0.84      0.92       166

    accuracy                           0.98      1115
   macro avg       0.99      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115

0.9766816143497757


La matriz de confusión muestra que el modelo realizó 949 predicciones correctas para la clase "ham" y 140 predicciones correctas para la clase "spam". No hubo 1 predicción incorrecta para "ham" y 26 predicciones incorrectas para "spam". Esto indica un alto rendimiento en la identificación de ambas clases, con un recall del 100% para "ham" y un recall del 84% para "spam". El accuracy del modelo es del 97.66%, lo que significa que clasifica correctamente aproximadamente el 98% de los mensajes en el conjunto de prueba.

Comparando estos resultados con el modelo Naive Bayes, el modelo Random Forest muestra un rendimiento generalmente superior. Tiene una precisión más alta para ambas clases ("ham" y "spam") y un recall más alto para la clase "ham". Esto sugiere que el modelo Random Forest es más efectivo en la identificación de mensajes de spam y no spam en comparación con el modelo Naive Bayes en este conjunto de datos específico.

### Predicciones con el Modelo Entrenado

Dado a que el modelo de RandomForest tuvo un mejor rendimiento prediciendo el conjunto de prueba, se usará este para las predicciones. Se creó la función **clasificador** en el archivo *spam_rf.py*. A continuación, se presentan algunas predicciones realizadas por el modelo entrenado para datos diferentes a los de entrenamiento:

In [7]:
from spam_rf import spam_clasificador

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\elian\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\elian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\elian\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!



1. **Ejemplo 1:**

   - Mensaje de Texto: "you are lucky, to claim you prize click here"
   - Predicción del Modelo de Random Forest: spam
   - Valor Real: "spam"
   

In [9]:
spam_clasificador("you are lucky, to claim you prize click here", classifier)

array(['spam'], dtype=object)

2. **Ejemplo 2:**

   - Mensaje de Texto: "Hello Elias, tell me when you arrive home, please"
   - Predicción del Modelo de Random Forest: ham
   - Valor Real: ham

In [10]:
spam_clasificador("Hello Elias, tell me when you arrive home, please", classifier)

array(['ham'], dtype=object)

3. **Ejemplo 3:**

   - Mensaje de Texto: "URGENT, we have been trying to contact you. Youve won over 900 thousend dollars, plese send you info to claim the money"
   - Predicción del Modelo de Random Forest: spam
   - Valor Real: spam

In [13]:
spam_clasificador("URGENT, we have been trying to contact you. Youve won over 900 thousend dollars, plese send you info to claim the money", classifier)

array(['spam'], dtype=object)